In [108]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam, RMSprop


In [109]:
dataframe = pd.read_csv("data/raw/DAX_Data.csv", usecols=[1])
dataframe = dataframe.dropna()
#plt.plot(dataframe)
dataframe.shape

(9213, 1)

In [110]:
#Convert pandas dataframe to numpy array
dataset = dataframe.values
dataset = dataset.astype('float32')
dataset.shape

(9213, 1)

In [111]:
scaler = MinMaxScaler(feature_range=(0, 1)) #Also try QuantileTransformer
dataset = scaler.fit_transform(dataset)
dataset.shape


(9213, 1)

In [112]:
train_size = int(len(dataset) * 0.66)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [113]:
def to_sequences(dataset, seq_size=1):
    x = []
    y = []

    for i in range(len(dataset)-seq_size-1):
        #print(i)
        window = dataset[i:(i+seq_size), 0]
        x.append(window)
        y.append(dataset[i+seq_size, 0])
        
    return np.array(x),np.array(y)

In [114]:
seq_size = 5  # Number of time steps to look back 
#Larger sequences (look further back) may improve forecasting.

trainX, trainY = to_sequences(train, seq_size)
testX, testY = to_sequences(test, seq_size)



print("Shape of training set: {}".format(trainX.shape))
print("Shape of test set: {}".format(testX.shape))

Shape of training set: (6074, 5)
Shape of test set: (3127, 5)


In [115]:
# Reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [116]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from scikeras.wrappers import KerasRegressor

def create_model(lstm_units=50, optimizer='adam'):
    model = Sequential()
    model.add(LSTM(64, input_shape=(None, seq_size), return_sequences=True))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dense(1))
    
    
    # Choose optimizer
    if optimizer == 'adam':
        opt = Adam()
    elif optimizer == 'sgd':
        opt = SGD()
    elif optimizer == 'rmsprop':
        opt = RMSprop()
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer}")
    
    model.compile(optimizer=opt, loss='mean_squared_error')
    return model


In [117]:
lstm_regressor = KerasRegressor(
    model=create_model,
    epochs=50,
    batch_size=6,
    verbose=0
)


In [118]:
param_grid = {
    'model__lstm_units': [50, 100, 150],    # Use 'model__' prefix to specify parameters for the model function
    'model__optimizer': ['adam', 'sgd', 'rmsprop'],
    'batch_size': [6, 12, 24],
    'epochs': [50, 100, 150]
}


In [119]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    estimator=lstm_regressor,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=3
)

grid_search_result = grid_search.fit(trainX, trainY)


ValueError: 
All the 243 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
243 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/riccardo/Desktop/Github/Exploring-the-power-of-rnns/venv1/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/riccardo/Desktop/Github/Exploring-the-power-of-rnns/venv1/lib/python3.10/site-packages/scikeras/wrappers.py", line 770, in fit
    self._fit(
  File "/Users/riccardo/Desktop/Github/Exploring-the-power-of-rnns/venv1/lib/python3.10/site-packages/scikeras/wrappers.py", line 925, in _fit
    X, y = self._initialize(X, y)
  File "/Users/riccardo/Desktop/Github/Exploring-the-power-of-rnns/venv1/lib/python3.10/si